# joyplot

## spatial data

In [6]:
import pandas as pd
import geopandas as gpd

In [4]:
# get wards, oa, lsoa, lookups etc
%run "0.0-lbth-spatial-datasets.ipynb"

In [9]:
# read in population density by oa dataset
try:
  popden_oa = pd.read_csv('../data/external/lbth_census_2021_pop_density_oa.csv')
except:
  pass
  # popden_oa = pd.read_csv('https://www.nomisweb.co.uk/api/v01/dataset/NM_2026_1.data.csv?date=latest&geography=629165479...629165982,629303674...629303812,629317322...629317326,629317336...629317343,629317349...629317360,629317362...629317366,629317371,629317374,629317376,629317378,629317379,629317381,629317385,629317388...629317392,629317394...629317397,629317399...629317403,629317405...629317407,629317409,629317411,629317412,629317416...629317420,629317422...629317424,629317426,629317429...629317434,629317436,629317437,629317440...629317442,629317444...629317450,629317452...629317456,629317459...629317461,629317463,629317466...629317468,629317472,629317474...629317479,629317481,629317483,629317486,629317487,629317490,629317492,629317494,629317495,629317497,629317499...629317502,629317504,629317505,629317507...629317509,629317512...629317514,629317517,629317520,629317523,629317525,629317527,629317531,629317534,629317535,629317537,629317538,629317540,629317543...629317548,629317551,629317554,629317555,629317558,629317560,629317562,629317563,629317565,629317566,629317569...629317573,629317576...629317578,629317580,629317582,629317584,629317585,629317587,629317590...629317592,629317594,629317596...629317599,629317601,629317603,629317604,629317606,629317607,629317610,629317612,629317613,629317615...629317617,629317619...629317621,629317623,629317625...629317629,629317631...629317634,629317636,629317640,629317648,629317650...629317659,629317662,629317663,629317666...629317669,629317672...629317687,629317689,629317691,629317694,629317695,629317697,629317698,629317700,629317701,629317703,629317704,629317706,629317707,629317711...629317714,629317716,629317718...629317720,629317722...629317724,629317726,629317729,629317731,629317733,629317736...629317742,629317744,629317746...629317748,629323624,629323625&cell=0&measures=20100')  popden_oa.to_csv('../data/external/lbth_census_2021_pop_density_oa.csv')

In [10]:
from shapely.geometry import LineString, Point
import math
import random
import numpy as np
import altair as alt

In [29]:
def renormalise(n, range1, range2): 
  """transpose a number from range1 to range2

  Args:
      n ([int]): [description]
      range1 ([list]): [dataset range]
      range2 ([list]): [latitude range]

  Returns:
      [float]: [recalibrated value]
  """
  delta1 = range1[1] - range1[0]
  delta2 = range2[1] - range2[0]
  return (delta2 * (n - range1[0]) / delta1) + range2[0]

def make_peak(point, val, range1, range2):
    """turn a spatial point into a data peak

    Args:
        point ([shapely.Point]): [i.e. a polygon centroid]
        val ([int]): [i.e. a value to represent]
        range1 ([list]): [min, max range of input data]
        range2 ([list]): [min, max range of output data]

    Returns:
        [shapely.LineString]: [peak sized by input val]
    """
    length = renormalise(val, range1, range2)
    start = Point(point.geometry.x - 0.001, point.geometry.y)
    angle = math.pi / 3
    mid = Point(point.geometry.x, point.geometry.y + length * math.sin(angle))
    end = Point(point.geometry.x + 0.001, point.geometry.y)
    line = LineString([start, mid, end])
    return line


In [32]:
%%capture
df = oa.merge(popden_oa, left_on='OA21CD', right_on='GEOGRAPHY_CODE')
df.geometry = df.to_crs(epsg=4326).centroid
df = df.sort_values(by=('OBS_VALUE'), ascending=True)
df['geometry'] = df.apply(lambda point : make_peak(point, point.OBS_VALUE, [df.OBS_VALUE.min(), df.OBS_VALUE.max()], [0, 0.01]), axis=1)

In [33]:
alt.Chart(df).mark_geoshape(
    fill='white',
    stroke='#888'
).project(
    "mercator"
).properties(
    width=500,
    height=500
)

alt.Chart(...)

In [ ]:
# try curving the lines
from shapely.geometry import Point, LineString, mapping
import numpy as np

fine_x = np.linspace(np.min(coords[:, 0]), np.max(coords[:, 0]), 1000)
fine_y = f(fine_x)

fine_coords = zip(fine_x, fine_y)
fine_line = LineString(fine_coords)

even_line = LineString([
    np.array(fine_line.interpolate(i))
    for i in np.arange(0, fine_line.length, 5) #points 5 units apart
])